In [16]:
import os
import tkinter as tk
from tkinter import Text,filedialog
from tkinter import *
import matplotlib.pyplot as plt
import random
import cv2
from PIL import Image,ImageTk
import numpy as np
from keras.models import load_model,save_model,Sequential
from keras.layers import Activation,Conv2D,Dense,Dropout,Flatten,MaxPooling2D
from keras.preprocessing import image
from keras.utils import to_categorical

In [ ]:
center = 112
upper_bound, lower_bound = 224, 0
pad = 10
region_ul, region_ll = 5000, 150
thresh_ul, thresh_ll = 255, 127
train_data = []
test_data = []
X_train = []
y_train = []
X_test = []
y_test = []
shape = [224,224,3]

In [ ]:
train_dir = 'Desktop/jupyter/skin_cancer_det/cancer data/train'
test_dir = 'Desktop/jupyter/skin_cancer_det/cancer data/test'
Categories = ['benign','malignant']

In [ ]:
def initialPrepare(img):
    blur = cv2.medianBlur(img,5)
    img = cv2.cvtColor(blur,cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    _,thresh = cv2.threshold(gray,thresh_ll,thresh_ul,cv2.THRESH_BINARY_INV)
    return thresh

In [ ]:
def findDist(x,y):
    X = (center-x)*(center-x)
    Y = (center-y)*(center-y)
    return X+Y

In [ ]:
def check(region_size):
    if(region_size<region_ul and region_size>region_ll):
            return True
    return False

In [ ]:
def centroid(contour):
    centroidXY = cv2.moments(contour)
    x = int(centroidXY['m10']/centroidXY['m00'])
    y = int(centroidXY['m01']/centroidXY['m00'])
    return x,y

In [ ]:
def finalRegion(curr_contour):
    perimeter = cv2.arcLength(curr_contour, True)
    box = cv2.approxPolyDP(curr_contour, 0.02 * perimeter, True)
    x , y , w, h = cv2.boundingRect(box)
    return x, y, w, h

In [ ]:
def show_images(y,X_images):
    c = 5
    r = 3
    fig = plt.figure(figsize=(12, 12))
    for i in range(1,r*c+1):
        ax = fig.add_subplot(r,c,i)
        if y[i] == 0:
            ax.title.set_text('Benign')
        else :
            ax.title.set_text('Malignant')
        plt.imshow(X_images[i], interpolation='nearest')
    plt.show()

In [ ]:
def preprocess(img):
    thresh=initialPrepare(img)
    _,contours,_ = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    max_val,curr_contour = 100000,contours[0]
    for contour in contours:
        region_size = cv2.contourArea(contour)
        ok = check(region_size)
        if(ok):
            x,y = centroid(contour)
            distance = findDist(x,y)
            if distance < max_val:
                curr_contour = contour
                max_val = distance  
    x_b ,y_b ,w ,h = finalRegion(curr_contour)
    x_f = max(lower_bound,x_b - pad)
    x_fb = min(upper_bound,x_b + w + pad)
    y_f = max(lower_bound,y_b - pad)
    y_fb = min(upper_bound,y_b + h + pad)

    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    final_img = img[y_f:y_fb,x_f:x_fb]
    final_img = cv2.resize(final_img,(224,224))
    return final_img

In [ ]:
def prepare_train_data():
    for category in Categories:
        path = os.path.join(train_dir,category)
        cat_idx = Categories.index(category)
        for img in os.listdir(path):
            try:
                image = cv2.imread(os.path.join(path,img))
                image = preprocess(image)
                train_data.append([image,cat_idx])
            except Exception as e:
                pass
prepare_train_data()

In [ ]:
random.shuffle(train_data)

In [ ]:
for features,labels in train_data:
    X_train.append(features)
    y_train.append(labels)

In [ ]:
X_train = np.array(X_train).reshape(-1,shape[0],shape[1],shape[2])
X_train = X_train/255
y_cat_train = to_categorical(y_train,2)

In [ ]:
show_images(y_train,X_train)

In [ ]:
def prepare_test_data():
    for category in Categories:
        path = os.path.join(test_dir,category)
        cat_idx = Categories.index(category)
        for img in os.listdir(path):
            try:
                image = cv2.imread(os.path.join(path,img))
                image = preprocess(image) 
                test_data.append([image,cat_idx])
            except Exception as e:
                pass
prepare_test_data()

In [ ]:
random.shuffle(test_data)

In [ ]:
for features,labels in test_data:
    X_test.append(features)
    y_test.append(labels)

In [ ]:
X_test = np.array(X_test).reshape(-1,shape[0],shape[1],shape[2])
X_test=X_test/255
y_cat_test = to_categorical(y_test,2)

In [ ]:
show_images(y_test,X_test)

In [ ]:
def train_model():
    model = Sequential()
    model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=(224,224,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=(224,224,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=(224,224,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(Dense(2,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(X_train,y_cat_train,epochs=5)
    return model
model = train_model()

In [ ]:
def eval_model(model,X_test,y_cat_test):
    model.evaluate(X_test,y_cat_test)
eval_model(model,X_test,y_cat_test)

In [ ]:
def save_model(model):
    model.save('Desktop/jupyter/skin_cancer_det/Skin_Cancer(Classification).h5')
save_model(model)

In [2]:
def load_processed_model():
    model = load_model('Desktop/jupyter/skin_cancer_det/Skin_Cancer(Classification).h5')
    return model

In [30]:
root = tk.Tk()
root.title('Skin Cancer Classification')
root.iconbitmap(r'D:\Photos & Videos\Logo.ico')
root.geometry("400x400")
canvas = Canvas(width=400, height=400, bg='black')
canvas.pack()

model = load_processed_model()

def Find():
    img_file = filedialog.askopenfilename(title='Select Image',filetypes=[("Image File",'.jpg')])
    img_display = Image.open(img_file)
    pic = ImageTk.PhotoImage(img_display)
    pic_label = Label(canvas,image = pic)
    pic_label.image=pic
    pic_label.pack()
    res = image.load_img(img_file,target_size=(224,224))
    res = image.img_to_array(res)
    res = np.expand_dims(res,axis=0)
    res = res/255
    output = model.predict(res)
    if(output[0][0]>output[0][1]):
        text1="There is a"
        print("There is a",output[0][0]*100,"chance that you dont have skin cancer")
    else:
        text1="There is a"
        print("There is a",output[0][1]*100,"chance that you have skin cancer")
openfile = tk.Button(root,text="Predict",padx=10,pady=5,fg="black",command = Find)
openfile.pack()

#text = tk.Text(root,height = 10,width=50)
#text.config(state="normal")
#text.insert(tk.INSERT,text1)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\coolb\Anaconda3\envs\python-cvcourse\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-30-5323e6c4aa74>", line 16, in Find
    pic_label = cv2.resize(pic_label,(400,400))
TypeError: src is not a numpy array, neither a scalar
